In [1]:
import pandas as pd
import numpy as np
import os
import sklearn.metrics
from sklearn.metrics import f1_score
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target
import multiprocessing
import itertools
from sklearn.ensemble import GradientBoostingClassifier as gbm 
from sklearn.cross_validation import KFold
import matplotlib.pylab as plt
from datetime import datetime, timedelta
import multiprocessing
from joblib import Parallel , delayed
from sklearn.metrics import precision_recall_fscore_support as score
import xgboost as xgb
os.getcwd()

'/Users/a0v0022/Documents/Logical_Purchase'

In [2]:
data = pd.read_csv("Logical_purchase_5500.csv",header=None)

In [3]:
data.columns = ['user_id','visit_date','product_id','units','dollars','cust_lead_date','cust_prod_lead_date','order_id','reordered','days_since_prior_order','cust_prod_aog','order_dow','week_day_name','cal_month_nbr','cal_month_name','cal_qtr_nbr','cal_year_nbr','cal_week_nbr','rank_cust','train_test']

In [4]:
data['days_since_prior_order'] = data['days_since_prior_order'].replace('\\N', 99999 )
data['days_since_prior_order'] =  data['days_since_prior_order'].apply(lambda x:float(x))
data['days_since_prior_order']= data['days_since_prior_order'].replace(99999 , np.NaN )

In [5]:
data['visit_date'] = data['visit_date'].replace('\\N', np.NaN )
data['visit_date'] =  pd.to_datetime(data['visit_date'])

In [6]:
LP_check = data[(data['user_id'] == 121800790)]
LP_check.to_csv('LP_check5.csv', index=False, header=True)

In [7]:
data['add_to_cart_order'] = data.groupby(['user_id', 'order_id'])['product_id'].rank(method='dense').astype(int)
data['eval_set'] = ['prior' if x > 1 else 'train' for x in data['train_test']]
data.rename(columns={'order_id': 'order_number'}, inplace=True)
data['order_id']= data.apply(lambda x:'%s_%s' % (x['user_id'],x['order_number']),axis=1)

In [8]:
data.columns

Index([u'user_id', u'visit_date', u'product_id', u'units', u'dollars',
       u'cust_lead_date', u'cust_prod_lead_date', u'order_number',
       u'reordered', u'days_since_prior_order', u'cust_prod_aog', u'order_dow',
       u'week_day_name', u'cal_month_nbr', u'cal_month_name', u'cal_qtr_nbr',
       u'cal_year_nbr', u'cal_week_nbr', u'rank_cust', u'train_test',
       u'add_to_cart_order', u'eval_set', u'order_id'],
      dtype='object')

In [9]:
for i in range(0,16):
    #Train Dataset
    init_date_train = min(data['visit_date'])+timedelta(days=i)
    last_date_train = init_date_train + timedelta(days=120)
    init_date_test = init_date_train + timedelta(days=15)
    last_date_test = last_date_train + timedelta(days=15)
    train_Y_date = last_date_train + timedelta(days=1)
    test_Y_date = last_date_test + timedelta(days=1)
    order_prior = data[data['visit_date'] >= init_date_train]
    order_prior = order_prior[order_prior['visit_date'] <= last_date_train]
    max(order_prior['visit_date'])
    order_train = data[data['visit_date'] == train_Y_date]
    order_train = order_train.loc[:,['user_id','product_id','reordered']].drop_duplicates()
    cust_map = data.loc[:,['user_id', 'rank_cust']]
    cust_map = cust_map.drop_duplicates()
    cust_map = pd.DataFrame(cust_map)
    orders = data.loc[: , ['user_id' ,'visit_date', 'order_id' ,'order_number', 'days_since_prior_order','eval_set', 'order_dow']]
    orders = orders.drop_duplicates()
    orders = pd.DataFrame(orders)
    cust_map.shape
    order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
    ['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})
    order_comsum.columns = ['user_id', 'order_number', 'days_since_prior_order_comsum']
    order_comsum = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]
    order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id']]
    order_product = pd.merge(order_product, order_comsum, on='order_id')
    temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
    temp = temp.to_frame('periods').reset_index()
    aggregated = temp.copy()
    aggregated['last'] = aggregated.periods.apply(lambda x: x[-1] if len(x) > 0 else np.nan)
    aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
    aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
    aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
    aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
    aggregated['sdev'] = aggregated.periods.apply(lambda x : np.std(x[:-1]))
    aggregated.drop('periods', axis=1, inplace=True)
    LP_check = aggregated[(aggregated['user_id'] == 121800790)]
    LP_check.to_csv('LP_check4.csv', index=False, header=True)
    prob = order_prior
    prob = prob.groupby(['product_id', 'user_id'],as_index=False).agg({'reordered':'sum', 'train_test': 'size'})
    prob.rename(columns={'sum': 'reordered', 'train_test': 'total'}, inplace=True)
    prob.reordered = (prob.reordered > 0).astype(np.float32)
    prob.total = (prob.total > 0).astype(np.float32)
    prob['reorder_prob'] = prob.reordered / prob.total
    prob = prob.groupby('product_id').agg({'reorder_prob': 'mean'}).rename(columns={'mean': 'reorder_prob'}).reset_index()
    prod_stat = order_prior.groupby('product_id').agg({'reordered': ['sum', 'size']})
    prod_stat.columns = prod_stat.columns.levels[1]
    prod_stat.rename(columns={'sum':'prod_reorders',
                              'size':'prod_orders'}, inplace=True)
    prod_stat.reset_index(inplace=True)
    prod_stat['reorder_ration'] = prod_stat['prod_reorders'] / prod_stat['prod_orders']
    prod_stat = pd.merge(prod_stat, prob, on='product_id')
    user_stat = orders.loc[orders.eval_set == 'prior', :].groupby('user_id').agg({'order_number': 'max',
                                                                                  'days_since_prior_order': ['sum',
                                                                                                             'mean',
                                                                                                             'median']})
    user_stat.columns = user_stat.columns.droplevel(0)
    user_stat.rename(columns={'max': 'user_orders',
                              'sum': 'user_order_starts_at',
                              'mean': 'user_mean_days_since_prior',
                              'median': 'user_median_days_since_prior'}, inplace=True)
    user_stat.reset_index(inplace=True)
    orders_products =  order_prior
    user_order_stat = orders_products.loc[order_prior.eval_set == 'prior', :].groupby('user_id').agg({'user_id': 'size',
                                                              'reordered': 'sum',
                                                              "product_id": lambda x: x.nunique()})
    user_order_stat.rename(columns={'user_id': 'user_total_products',
                                    'product_id': 'user_distinct_products',
                                    'reordered': 'user_reorder_ratio'}, inplace=True)
    user_order_stat.reset_index(inplace=True)
    user_order_stat.user_reorder_ratio = user_order_stat.user_reorder_ratio / user_order_stat.user_total_products
    user_stat = pd.merge(user_stat, user_order_stat, on='user_id')
    user_stat['user_average_basket'] = user_stat.user_total_products / user_stat.user_orders
    prod_usr = order_prior.groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
    prod_usr.rename(columns={'user_id':'prod_users_unq'}, inplace=True)
    prod_usr.reset_index(inplace=True)
    prod_usr_reordered = order_prior.loc[order_prior.reordered, :].groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
    prod_usr_reordered.rename(columns={'user_id': 'prod_users_unq_reordered'}, inplace=True)
    prod_usr_reordered.reset_index(inplace=True)
    orders_products = order_prior
    order_stat = orders_products.groupby('order_id').agg({'order_id': 'size'}) \
        .rename(columns={'order_id': 'order_size'}).reset_index()

    orders_products = pd.merge(orders_products, order_stat, on='order_id')
    data1 = orders_products.groupby(['user_id', 'product_id']).agg({'user_id': 'size',
                                                                   'order_number': ['min', 'max'],
                                                                   'days_since_prior_order': ['mean', 'median'],
                                                                   'order_dow': ['mean', 'median'],
                                                                   'reordered': ['sum']})
    data1.columns = data1.columns.droplevel(0)
    data1.columns = [ 'up_first_order', 'up_last_order', 'order_dow_mean', 'order_dow_median','up_orders',
                    'days_since_prior_order_mean', 'days_since_prior_order_median', 'reordered_sum']
    data1['user_product_reordered_ratio'] = (data1.reordered_sum + 1.0) / data1.up_orders
    data1.reset_index(inplace=True)

    data1 = pd.merge(data1, prod_stat, on='product_id')
    data1 = pd.merge(data1, user_stat, on='user_id')

    data1['up_order_rate'] = data1.up_orders / data1.user_orders
    data1['up_orders_since_last_order'] = data1.user_orders - data1.up_last_order
    data1['up_order_rate_since_first_order'] = data1.user_orders / (data1.user_orders - data1.up_first_order + 1)
    orders1 = orders[orders['visit_date'] >= init_date_train]
    orders1 = orders1[orders1['visit_date'] <= last_date_train]
    orders_user = orders1[['order_id', 'user_id']]
    labels = pd.merge(order_prior, orders_user, on=['user_id','order_id'])
    labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()
    train = pd.merge(labels, prod_usr, on='product_id')
    train = pd.merge(train, prod_usr_reordered, on='product_id', how='left')
    train.prod_users_unq_reordered.fillna(0, inplace=True)
    train = pd.merge(train, data1, on=['product_id', 'user_id'])
    train = pd.merge(train, aggregated, on=['user_id',  'product_id'])
    LP_check = train[(train['user_id'] == 121800790)]
    LP_check.to_csv('LP_check6.csv', index=False, header=True)
    dep = order_train.loc[:,['user_id','product_id','reordered']]
    train = pd.merge(train, dep,on=['user_id',  'product_id'], how='left')
    train.reordered.fillna(0, inplace=True)
    train.fillna(999, inplace=True)
    train = pd.merge(train, cust_map, on='user_id')
    train.shape
    LP_check = train[(train['user_id'] == 121800790)]
    LP_check.to_csv('LP_check7.csv', index=False, header=True)
    train["counter"]=i
    if i==0:
        Base_train=train
    else :
        Base_train = Base_train.append(train,ignore_index=True)
       
    #Test Data Creation
    
    order_prior = data[data['visit_date'] >= init_date_test]
    order_prior = order_prior[order_prior['visit_date'] <= last_date_test]
    min(order_prior['visit_date'])
    order_train = data[data['visit_date'] == test_Y_date]
    order_train = order_train.loc[:,['user_id','product_id','reordered']].drop_duplicates()
    data.columns
    cust_map = data.loc[:,['user_id', 'rank_cust']]
    cust_map = cust_map.drop_duplicates()
    cust_map = pd.DataFrame(cust_map)
    orders = data.loc[: , ['user_id' ,'visit_date', 'order_id' ,'order_number', 'days_since_prior_order','eval_set', 'order_dow']]
    orders = orders.drop_duplicates()
    orders = pd.DataFrame(orders)
    cust_map.shape
    order_comsum = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])\
    ['days_since_prior_order'].sum().groupby(level=[0]).cumsum().reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})
    order_comsum.columns = ['user_id', 'order_number', 'days_since_prior_order_comsum']
    order_comsum = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]
    order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id']]
    order_product = pd.merge(order_product, order_comsum, on='order_id')
    temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))
    temp = temp.to_frame('periods').reset_index()
    aggregated = temp.copy()
    aggregated['last'] = aggregated.periods.apply(lambda x: x[-1] if len(x) > 0 else np.nan)
    aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)
    aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)
    aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))
    aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))
    aggregated['sdev'] = aggregated.periods.apply(lambda x : np.std(x[:-1]))
    aggregated.drop('periods', axis=1, inplace=True)
    prob = order_prior
    prob = prob.groupby(['product_id', 'user_id'],as_index=False).agg({'reordered':'sum', 'train_test': 'size'})
    prob.rename(columns={'sum': 'reordered', 'train_test': 'total'}, inplace=True)
    prob.reordered = (prob.reordered > 0).astype(np.float32)
    prob.total = (prob.total > 0).astype(np.float32)
    prob['reorder_prob'] = prob.reordered / prob.total
    prob = prob.groupby('product_id').agg({'reorder_prob': 'mean'}).rename(columns={'mean': 'reorder_prob'}).reset_index()
    prod_stat = order_prior.groupby('product_id').agg({'reordered': ['sum', 'size']})
    prod_stat.columns = prod_stat.columns.levels[1]
    prod_stat.rename(columns={'sum':'prod_reorders',
                              'size':'prod_orders'}, inplace=True)
    prod_stat.reset_index(inplace=True)
    prod_stat['reorder_ration'] = prod_stat['prod_reorders'] / prod_stat['prod_orders']
    prod_stat = pd.merge(prod_stat, prob, on='product_id')
    user_stat = orders.loc[orders.eval_set == 'prior', :].groupby('user_id').agg({'order_number': 'max',
                                                                                  'days_since_prior_order': ['sum',
                                                                                                             'mean',
                                                                                                             'median']})
    user_stat.columns = user_stat.columns.droplevel(0)
    user_stat.rename(columns={'max': 'user_orders',
                              'sum': 'user_order_starts_at',
                              'mean': 'user_mean_days_since_prior',
                              'median': 'user_median_days_since_prior'}, inplace=True)
    user_stat.reset_index(inplace=True)
    orders_products =  order_prior
    user_order_stat = orders_products.loc[order_prior.eval_set == 'prior', :].groupby('user_id').agg({'user_id': 'size',
                                                              'reordered': 'sum',
                                                              "product_id": lambda x: x.nunique()})
    user_order_stat.rename(columns={'user_id': 'user_total_products',
                                    'product_id': 'user_distinct_products',
                                    'reordered': 'user_reorder_ratio'}, inplace=True)
    user_order_stat.reset_index(inplace=True)
    user_order_stat.user_reorder_ratio = user_order_stat.user_reorder_ratio / user_order_stat.user_total_products
    user_stat = pd.merge(user_stat, user_order_stat, on='user_id')
    user_stat['user_average_basket'] = user_stat.user_total_products / user_stat.user_orders
    prod_usr = order_prior.groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
    prod_usr.rename(columns={'user_id':'prod_users_unq'}, inplace=True)
    prod_usr.reset_index(inplace=True)
    prod_usr_reordered = order_prior.loc[order_prior.reordered, :].groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
    prod_usr_reordered.rename(columns={'user_id': 'prod_users_unq_reordered'}, inplace=True)
    prod_usr_reordered.reset_index(inplace=True)
    orders_products = order_prior
    order_stat = orders_products.groupby('order_id').agg({'order_id': 'size'}) \
        .rename(columns={'order_id': 'order_size'}).reset_index()

    orders_products = pd.merge(orders_products, order_stat, on='order_id')
    data1 = orders_products.groupby(['user_id', 'product_id']).agg({'user_id': 'size',
                                                                   'order_number': ['min', 'max'],
                                                                   'days_since_prior_order': ['mean', 'median'],
                                                                   'order_dow': ['mean', 'median'],
                                                                   'reordered': ['sum']})
    data1.columns = data1.columns.droplevel(0)
    data1.columns = [ 'up_first_order', 'up_last_order', 'order_dow_mean', 'order_dow_median','up_orders',
                    'days_since_prior_order_mean', 'days_since_prior_order_median', 'reordered_sum']
    data1['user_product_reordered_ratio'] = (data1.reordered_sum + 1.0) / data1.up_orders
    data1.reset_index(inplace=True)

    data1 = pd.merge(data1, prod_stat, on='product_id')
    data1 = pd.merge(data1, user_stat, on='user_id')

    data1['up_order_rate'] = data1.up_orders / data1.user_orders
    data1['up_orders_since_last_order'] = data1.user_orders - data1.up_last_order
    data1['up_order_rate_since_first_order'] = data1.user_orders / (data1.user_orders - data1.up_first_order + 1)
    orders1 = orders[orders['visit_date'] >= init_date_test]
    orders1 = orders1[orders1['visit_date'] <= last_date_test]
    orders_user = orders1[['order_id', 'user_id']]
    labels = pd.merge(order_prior, orders_user, on=['user_id','order_id'])
    labels = labels.loc[:, ['user_id', 'product_id']].drop_duplicates()
    test = pd.merge(labels, prod_usr, on='product_id')
    test = pd.merge(test, prod_usr_reordered, on='product_id', how='left')
    test.prod_users_unq_reordered.fillna(0, inplace=True)
    test = pd.merge(test, data1, on=['product_id', 'user_id'])
    test = pd.merge(test, aggregated, on=['user_id',  'product_id'])
    dep = order_train.loc[:,['user_id','product_id','reordered']]
    test = pd.merge(test, dep,on=['user_id',  'product_id'], how='left')
    test.reordered.fillna(0, inplace=True)
    test.fillna(999, inplace=True)
    test = pd.merge(test, cust_map, on='user_id')
    test.shape
    LP_check = test[(test['user_id'] == 121800790)]
    LP_check.to_csv('LP_check8.csv', index=False, header=True)
    test["counter"]=i
    if i==0:
        Base_test=test
    else :
        Base_test = Base_test.append(test,ignore_index =True)  

/Users/a0v0022/anaconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/a0v0022/anaconda/lib/python2.7/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)


In [10]:
Base_train.shape

(25841251, 37)

In [11]:
Base_test.shape

(25894409, 37)

In [12]:
Base_train.to_csv('Base_train.csv', index=False, header=True)
Base_test.to_csv('Base_test.csv', index=False, header=True)

In [13]:
LP_check = Base_train[(Base_train['user_id'] == 121800790)]
LP_check.to_csv('LP_check0.csv', index=False, header=True)

In [14]:
train1 = Base_train[(Base_train['counter'] == 0)]

In [15]:
train1.shape

(1615609, 37)

In [86]:
train_X = train.iloc[:, 2:34]
test_X = test.iloc[:, 2:34]

In [87]:
feature_names = list(train_X.columns)

In [88]:
feature_names

['prod_users_unq',
 'prod_users_unq_reordered',
 'up_first_order',
 'up_last_order',
 'order_dow_mean',
 'order_dow_median',
 'up_orders',
 'days_since_prior_order_mean',
 'days_since_prior_order_median',
 'reordered_sum',
 'user_product_reordered_ratio',
 'prod_reorders',
 'prod_orders',
 'reorder_ration',
 'reorder_prob',
 'user_order_starts_at',
 'user_mean_days_since_prior',
 'user_median_days_since_prior',
 'user_orders',
 'user_total_products',
 'user_distinct_products',
 'user_reorder_ratio',
 'user_average_basket',
 'up_order_rate',
 'up_orders_since_last_order',
 'up_order_rate_since_first_order',
 'last',
 'prev1',
 'prev2',
 'median',
 'mean',
 'sdev']

In [89]:
train_X = train_X.as_matrix()
test_X = test_X.as_matrix()

In [90]:
not_mask

array([ True, False,  True, ...,  True,  True,  True], dtype=bool)

In [91]:
train_Y = train['reordered']
test_Y = test['reordered']

In [92]:
mask = np.random.choice([False, True], len(train_X), p=[0.75, 0.25])
not_mask = ~mask

In [93]:
xg_train = xgb.DMatrix(train_X[not_mask], label=train_Y[not_mask], feature_names=feature_names)
xg_val = xgb.DMatrix(train_X[mask], label=train_Y[mask], feature_names=feature_names)
xg_test = xgb.DMatrix(test_X , feature_names=feature_names)

param = {}
param['objective'] = 'binary:logistic'
param['eta'] = 0.05
param['max_depth'] = 6
param['silent'] = 1
param['eval_metric'] = "logloss"
param['subsample'] = 0.75
param['colsample_bytree'] = 0.75

In [74]:
cv = xgb.cv(param, xg_train, 5, nfold=5, early_stopping_rounds=5, verbose_eval=1)

[0]	train-logloss:0.646596+1.26079e-05	test-logloss:0.646602+2.17348e-05
[1]	train-logloss:0.604479+1.31757e-05	test-logloss:0.604489+5.2458e-05
[2]	train-logloss:0.566198+2.54527e-05	test-logloss:0.566213+7.13033e-05
[3]	train-logloss:0.531267+4.01368e-05	test-logloss:0.531288+8.90092e-05
[4]	train-logloss:0.499294+2.28263e-05	test-logloss:0.499321+0.000126563


In [94]:
evallist  = [(xg_val, 'eval')]
bst = xgb.train( param, xg_train, 5, evallist )

[0]	eval-logloss:0.646592
[1]	eval-logloss:0.604488
[2]	eval-logloss:0.5662
[3]	eval-logloss:0.531269
[4]	eval-logloss:0.49927


In [97]:
feature_imp = pd.DataFrame(bst.get_fscore().items(), columns=['feature','importance'])
feature_imp.sort('importance', ascending=False)

,feature,importance
19,order_dow_mean,28
12,order_dow_median,27
22,user_mean_days_since_prior,17
0,up_orders,16
4,user_reorder_ratio,16
1,user_average_basket,14
23,median,12
10,user_distinct_products,11
24,up_order_rate,10
21,last,8


In [16]:
preds = bst.predict( xg_test )

In [31]:
prediction = pd.DataFrame(preds)
prediction.columns = ['reordered_p']
Submit = test.iloc[:,(0,1,34)]
submit1 = pd.concat([Submit, prediction], axis=1)

In [32]:
f1optimization = submit1
f1optimization['rank'] = f1optimization.groupby(['user_id'])['reordered_p'].rank(method = 'dense' , ascending = False).astype(int)

In [34]:
def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

In [35]:
class F1Optimizer():
    def __init__(self):
        pass

    @staticmethod
    def get_expectations(P, pNone=None):
        expectations = []
        P = np.sort(P)[::-1]

        n = np.array(P).shape[0]
        DP_C = np.zeros((n + 2, n + 1))
        if pNone is None:
            pNone = (1.0 - P).prod()

        DP_C[0][0] = 1.0
        for j in range(1, n):
            DP_C[0][j] = (1.0 - P[j - 1]) * DP_C[0, j - 1]

        for i in range(1, n + 1):
            DP_C[i, i] = DP_C[i - 1, i - 1] * P[i - 1]
            for j in range(i + 1, n + 1):
                DP_C[i, j] = P[j - 1] * DP_C[i - 1, j - 1] + (1.0 - P[j - 1]) * DP_C[i, j - 1]

        DP_S = np.zeros((2 * n + 1,))
        DP_SNone = np.zeros((2 * n + 1,))
        for i in range(1, 2 * n + 1):
            DP_S[i] = 1. / (1. * i)
            DP_SNone[i] = 1. / (1. * i + 1)
        for k in range(n + 1)[::-1]:
            f1 = 0
            f1None = 0
            for k1 in range(n + 1):
                f1 += 2 * k1 * DP_C[k1][k] * DP_S[k + k1]
                f1None += 2 * k1 * DP_C[k1][k] * DP_SNone[k + k1]
            for i in range(1, 2 * k - 1):
                DP_S[i] = (1 - P[k - 1]) * DP_S[i] + P[k - 1] * DP_S[i + 1]
                DP_SNone[i] = (1 - P[k - 1]) * DP_SNone[i] + P[k - 1] * DP_SNone[i + 1]
            expectations.append([f1None + 2 * pNone / (2 + k), f1])

        return np.array(expectations[::-1]).T

    @staticmethod
    def maximize_expectation(P, pNone=None):
        expectations = F1Optimizer.get_expectations(P, pNone)

        ix_max = np.unravel_index(expectations.argmax(), expectations.shape)
        max_f1 = expectations[ix_max]

        predNone = True if ix_max[0] == 0 else False
        best_k = ix_max[1]

        return best_k, predNone, max_f1

    @staticmethod
    def _F1(tp, fp, fn):
        return 2 * tp / (2 * tp + fp + fn)

    @staticmethod
    def _Fbeta(tp, fp, fn, beta=1.0):
        beta_squared = beta ** 2
        return (1.0 + beta_squared) * tp / ((1.0 + beta_squared) * tp + fp + beta_squared * fn)


def print_best_prediction(P, pNone=None):
    #print("Maximize F1-Expectation")
    #print("=" * 23)
    P = np.sort(P)[::-1]
    n = P.shape[0]
    L = ['L{}'.format(i + 1) for i in range(n)]

    if pNone is None:
        #print("Estimate p(None|x) as (1-p_1)*(1-p_2)*...*(1-p_n)")
        pNone = (1.0 - P).prod()

    PL = ['p({}|x)={}'.format(l, p) for l, p in zip(L, P)]
    #print("Posteriors: {} (n={})".format(PL, n))
    #print("p(None|x)={}".format(pNone))

    opt = F1Optimizer.maximize_expectation(P, pNone)
    best_prediction = ['None'] if opt[1] else []
    best_prediction += (L[:opt[0]])
    f1_max = opt[2]
    k_max = opt[0]
    pred_none = opt[1]
    #print("Prediction {} yields best E[F1] of {} with max_k of {} and pred_none is {} \n".format(best_prediction, f1_max , k_max,pred_none ))
    return k_max, pred_none
    

In [36]:
def create_products(df):
    prob = df.reordered_p.values
    sort_index = np.argsort(prob)[::-1]
    values = print_best_prediction(df.reordered_p.values)
    df.loc[:, 'k_max'] = values[0]
    df.loc[:, 'pred_none'] = values[1]
    return df

In [37]:
data1 = applyParallel(f1optimization.groupby(f1optimization.user_id) , create_products).reset_index()

In [38]:
LP_check = data1[(data1['user_id'] == 121800790)]
LP_check.to_csv('LP_check9.csv', index=False, header=True)

In [27]:
submit1['Reorder_predict'] = [1 if x > 0.5 else 0 for x in submit1['Probab']]

In [28]:
predicted = submit1['Reorder_predict'] 
y_test = test['reordered']

In [29]:
precision, recall, fscore, support = score(y_test, predicted)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [ 0.98546605  0.01972387]
recall: [  9.99373664e-01   8.54299261e-04]
fscore: [ 0.99237113  0.00163767]
support: [1587007   23411]
